In [55]:
from tda import auth, client
from dotenv import load_dotenv
from selenium import webdriver
from pathlib import Path
import psycopg2
import pandas as pd
import numpy as np
import json
import os

In [9]:
def connect_client(api_key, redirect_uri, token_path, webdriver_func):
    client = auth.easy_client(
        api_key = api_key,
        redirect_uri = REDIRECT_URI,
        token_path = TOKEN_PATH,
        webdriver_func = webdriver_func,
    )
    return client

def get_webdriver(path = None):
    if not path:
        path = Path(__file__).parents[1].absolute()
        path = os.path.join(path, 'chromedriver')
    return webdriver.Chrome(path)

In [10]:
load_dotenv()

# Get .env variables
TD_KEY = os.getenv('CONSUMER_KEY')
ACC_NUMBER = os.getenv('ACC_NUMBER')
REDIRECT_URI = os.getenv('REDIRECT_URI')
TOKEN_PATH = os.path.join('../tokens/token.pickle')
API_KEY = TD_KEY + '@AMER.OAUTHAP'
CLIENT = connect_client(API_KEY, REDIRECT_URI, TOKEN_PATH, get_webdriver)

In [11]:
# Import data
etf_list = pd.read_excel('../exempt_etfs.xlsx')
print(etf_list)

    Ticker
0     ZYUS
1      QUS
2     GVAL
3     IEUR
4     ASHR
..     ...
265   RESE
266    DEM
267   DXJS
268    DON
269    DES

[270 rows x 1 columns]


In [73]:
full_array = etf_list.to_numpy().ravel()
full_list = full_array.tolist()
query = ','.join(full_list)
response = CLIENT.get_quotes(','.join(full_list)).json()

'''
for index, row in etf_list.iterrows():
    ticker = row['Ticker']
    response = CLIENT.get_quote(ticker).json()
    if len(response) > 0:
        sec_att = {
            'symbol': response[ticker]['symbol'],
            'description': response[ticker]['description'],
            'asset_type': response[ticker]['asset_type'],
            'main_type': response[ticker]['main_type'],
        }
        tda_supported.append(row['Ticker'])
'''
print(json.dumps(response, indent = 4))

{
    "QUS": {
        "assetType": "ETF",
        "assetMainType": "EQUITY",
        "cusip": "78468R812",
        "assetSubType": "ETF",
        "symbol": "QUS",
        "description": "SPDR MSCI USA StrategicFactors ETF",
        "bidPrice": 78.77,
        "bidSize": 0,
        "bidId": " ",
        "askPrice": 120.0,
        "askSize": 500,
        "askId": "K",
        "lastPrice": 97.58,
        "lastSize": 100,
        "lastId": "P",
        "openPrice": 97.08,
        "highPrice": 97.58,
        "lowPrice": 96.82,
        "bidTick": " ",
        "closePrice": 96.8539,
        "netChange": 0.7261,
        "totalVolume": 17546,
        "quoteTimeInLong": 1598644808292,
        "tradeTimeInLong": 1598645400003,
        "mark": 97.58,
        "exchange": "p",
        "exchangeName": "PACIFIC",
        "marginable": true,
        "shortable": true,
        "volatility": 0.0571,
        "digits": 2,
        "52WkHigh": 99.41,
        "52WkLow": 64.57,
        "nAV": 0.0,
        "peR

In [54]:
tda_supported = list()

for ticker in response:
    ticker_att = [
        response[ticker]['symbol'],
        response[ticker]['description'],
        response[ticker]['assetType'],
        response[ticker]['assetMainType'],
    ]
    tda_supported.append(ticker_att)
    
tda_supported_df = pd.DataFrame(tda_supported, columns = ['symbol', 'description', 'asset_type','main_type'])
tda_supported_df.to_csv('../tda_supported_tickers.csv')
len(tda_supported)

216

In [56]:
conn = psycopg2.connect("dbname = historical user=postgres")
cur = conn.cursor()

In [71]:
args_str = list()

for index, row in tda_supported_df.iterrows():
    entry = cur.mogrify("(%s, %s, %s, %s)", (row['symbol'], row['description'], row['asset_type'], row['main_type']))
    entry = entry.decode('ASCII')
    args_str.append(entry)
    
args_str = ','.join(args_str)
pg_query = "INSERT INTO securities (symbol, description, asset_type, main_type) VALUES " + args_str + ";"
cur.execute(pg_query)
conn.commit()

In [72]:
cur.close()
conn.close()